In [1]:
import os
import json
import logging
from datetime import datetime

In [4]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/harsh_arma/.config/sagemaker/config.yaml


In [5]:
boto3.set_stream_logger(name="botocore.credentials", level=logging.WARNING)


In [7]:
def check_aws_configuration():
    try:
        # Attempt to create an AWS S3 client
        s3 = boto3.client('s3')
        
        # List buckets to verify AWS credentials and region
        buckets = s3.list_buckets()
        
        print("AWS Configuration is successful.")
        print("Available S3 Buckets:")
        for bucket in buckets['Buckets']:
            print(f"- {bucket['Name']}")
        
        return True
    except Exception as e:
        print(f"Error: {e}")
        print("AWS Configuration is not successful.")
        return False

# Check AWS configuration
check_aws_configuration()

AWS Configuration is successful.
Available S3 Buckets:
- airbyte-data-bucket
- apple-maps-rank-data
- aws-cloudtrail-logs-173829655778-1bc62e66
- aws-sam-cli-managed-default-samclisourcebucket-ujqrexj3n4l8
- cf-templates-1f8yr71fcychj-us-east-1
- chat-widget-dev-serverlessdeploymentbucket-c2tv25twqqud
- chat-widget-local-serverlessdeploymentbucket-5d5l8p5zoshf
- chrone-app-chat-campaign-jsons
- chrone-biz-keyword-jsons
- chrone-biz-react-website-dev
- chrone-biz-website-react
- chrone-community-data
- chrone-email-campaign-jsons
- chrone-global-assets
- chrone-internal-media-generation-reactjs
- chrone-lambda-layers-zip
- chrone-lead-magnet
- chrone-lead-magnet-dev
- chrone-ml-models
- chrone-public-media
- chrone-sales-rank-extraction-jsons
- chrone-sms-campaign-jsons
- chrone-sp-website
- chrone-sp-website-internal
- chrone-sp-website-media-bank
- chrone-sp-website-pages
- chrone-ui-reactjs
- chrone-website
- chrone-website-edit-application
- chrone.ai
- chrone.app
- chrone.biz
- chr

True

In [10]:
# sess = sagemaker.Session()
sess = sagemaker.Session(boto_session=boto3.Session(region_name='us-east-1'))
region = sess.boto_region_name
print(region)

us-east-1


In [16]:
role_arn = "arn:aws:iam::173829655778:user/harsh_sharma"

In [18]:
bucket_name = "imagetaggingdemo"
data_path = "img_clf_multilabel_lst"

nclasses = 3
nimgs_train = 11
nepochs = 10
mini_batch_size = 3

train_instance_type = "ml.g4dn.xlarge"
job_name_prefix = "img-clf-multilabel"

In [19]:
train_image_uri = sagemaker.image_uris.retrieve(
    framework="image-classification",
    region=region,
    image_scope="training",
    version="latest",
)
print(train_image_uri)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.


811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:1


In [20]:
s3_output_path = "s3://{}/{}/{}".format(bucket_name, data_path, "model_output")


In [22]:
clf_estimator = sagemaker.estimator.Estimator(
    image_uri=train_image_uri,
    role=role_arn,
    instance_count=1,
    instance_type=train_instance_type,
    volume_size=50,
    max_run=360000,
    input_mode="File",
    output_path=s3_output_path,
    sagemaker_session=sess,
)

In [23]:
clf_estimator.set_hyperparameters(
    num_classes=nclasses,  # update this
    epochs=nepochs,  # update this
    num_training_samples=nimgs_train,  # update this
    mini_batch_size=mini_batch_size,  # update this
    num_layers=18,
    use_pretrained_model=1,
    image_shape="3,224,224",
    resize=256,
    learning_rate=0.001,
    use_weighted_loss=1,
    augmentation_type="crop_color_transform",
    precision_dtype="float32",
    multi_label=0,
)

In [24]:
s3_train_imgs = "s3://{}/{}/{}".format(bucket_name, data_path, "train_imgs")
s3_valid_imgs = "s3://{}/{}/{}".format(bucket_name, data_path, "valid_imgs")
s3_train_annot = "s3://{}/{}/{}".format(bucket_name, data_path, "train_annots")
s3_valid_annot = "s3://{}/{}/{}".format(bucket_name, data_path, "valid_annots")

train_imgs = sagemaker.inputs.TrainingInput(
    s3_train_imgs,
    distribution="FullyReplicated",
    content_type="application/jpg",
    s3_data_type="S3Prefix",
)
valid_imgs = sagemaker.inputs.TrainingInput(
    s3_valid_imgs,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)
train_annot = sagemaker.inputs.TrainingInput(
    s3_train_annot,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)
valid_annot = sagemaker.inputs.TrainingInput(
    s3_valid_annot,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)

data_channels = {
    "train": train_imgs,
    "validation": valid_imgs,
    "train_lst": train_annot,
    "validation_lst": valid_annot,
}

In [25]:
timestamp = (
    str(datetime.now().replace(microsecond=0)).replace(" ", "-").replace(":", "-")
)
job_name = job_name_prefix + timestamp
print(job_name)

img-clf-multilabel2023-12-14-15-47-20


In [26]:
clf_estimator.fit(inputs=data_channels, logs=True, job_name=job_name)


INFO:sagemaker:Creating training-job with name: img-clf-multilabel2023-12-14-15-47-20


ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: 1 validation error detected: Value 'arn:aws:iam::173829655778:user/harsh_sharma' at 'roleArn' failed to satisfy constraint: Member must satisfy regular expression pattern: ^arn:aws[a-z\-]*:iam::\d{12}:role/?[a-zA-Z_0-9+=,.@\-_/]+$